<a href="https://colab.research.google.com/github/JJHyungo/MyPrivate/blob/master/RNN_Helloworld_Jinho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn
import numpy as np

In [0]:
# # input_text = 'hello'
# input_text = ['hello']
# # text = ['hello']

# # Join all the sentences together and extract the unique characters from the combined sentences
# chars = set(''.join(input_text))
# print(chars)
# # Creating a dictionary that maps integers to the characters
# int2char = dict(enumerate(chars))
# print(int2char)

# # Creating another dictionary that maps characters to integers
# char2int = {char: ind for ind, char in int2char.items()}
# print(char2int)

# # Creating lists that will hold our input and target sequences
# input_seq = []
# target_seq = []

# input_seq = input_text[:-1]
# target_seq = input_text[1:]
# print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq, target_seq))

input_text = ['hey how are you','good i am fine','have a nice day']
# input_text = ['hello']

# Join all the sentences together and extract the unique characters from the combined sentences
chars = sorted(set(''.join(input_text)))
print(chars)

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))
print(int2char)

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}
print(char2int)

[' ', 'a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'm', 'n', 'o', 'r', 'u', 'v', 'w', 'y']
{0: ' ', 1: 'a', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'm', 10: 'n', 11: 'o', 12: 'r', 13: 'u', 14: 'v', 15: 'w', 16: 'y'}
{' ': 0, 'a': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'm': 9, 'n': 10, 'o': 11, 'r': 12, 'u': 13, 'v': 14, 'w': 15, 'y': 16}


In [0]:
maxlen = len(max(input_text, key=len))
print("The longest string has {} characters".format(maxlen))

The longest string has 15 characters


In [0]:
# Padding

# A simple loop that loops through the list of sentences and   
# adds a ' ' whitespace until the length of the sentence matches  
# the length of the longest sentence  
for i in range(len(input_text)):
    while len(input_text[i])<maxlen:
        input_text[i] += ' '

In [0]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(input_text)):
    # Remove last character for input sequence
    input_seq.append(input_text[i][:-1])
    
    # Remove firsts character for target sequence
    target_seq.append(input_text[i][1:])
    print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))



Input Sequence: hey how are yo
Target Sequence: ey how are you
Input Sequence: good i am fine
Target Sequence: ood i am fine 
Input Sequence: have a nice da
Target Sequence: ave a nice day


In [0]:

for i in range(len(input_text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]
    print(input_seq[i])
    print(target_seq[i])

[7, 4, 16, 0, 7, 11, 15, 0, 1, 12, 4, 0, 16, 11]
[4, 16, 0, 7, 11, 15, 0, 1, 12, 4, 0, 16, 11, 13]
[6, 11, 11, 3, 0, 8, 0, 1, 9, 0, 5, 8, 10, 4]
[11, 11, 3, 0, 8, 0, 1, 9, 0, 5, 8, 10, 4, 0]
[7, 1, 14, 4, 0, 1, 0, 10, 8, 2, 4, 0, 3, 1]
[1, 14, 4, 0, 1, 0, 10, 8, 2, 4, 0, 3, 1, 16]


In [0]:


dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(input_text)
# batch_size = 1
print("dict_size : {}" .format(dict_size))
print("seq_len : {}" .format(seq_len))
print("batch_size : {}" .format(batch_size))

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features




dict_size : 17
seq_len : 14
batch_size : 3


In [0]:
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))
# print(input_seq)

Input shape: (3, 14, 17) --> (Batch Size, Sequence Length, One-Hot Encoding Size)


In [0]:


input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

In [0]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [0]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        print("forward x.shape :" + str(x.shape))
        print("forward rnn hidden.shape :" + str(hidden.shape))
        print("forward rnn out.shape : " + str(out.shape))
    
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        print("forward fc out.shape : " + str(out.shape))
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

In [0]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
print(model)
# We'll also set the model to the device that we defined earlier (default is CPU)
model.to(device)

# Define hyperparameters
n_epochs = 5
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Model(
  (rnn): RNN(17, 12, batch_first=True)
  (fc): Linear(in_features=12, out_features=17, bias=True)
)


tensor([[-0.0017,  0.1453,  0.9029, -1.4500,  0.2217],
        [ 0.3834, -1.3571,  0.0868, -0.0920, -0.7467],
        [ 0.7457, -1.1687,  1.0527,  0.6568,  1.2617]], requires_grad=True)
tensor([2, 4, 0])


In [0]:
# Training Run
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    print("training input_seq.shape : " + str(input_seq.shape))
    input_seq = input_seq.to(device)
    output, hidden = model(input_seq)
    print("training target_seq.shape :" + str(target_seq.shape)) # tensor([[0., 2., 2., 3.]])
    # https://pytorch.org/docs/stable/nn.html -> explain shape of nn.CrossEntropyLoss()
    """
    # temp_input = torch.randn(3, 5, requires_grad=True)
    # temp_target = torch.empty(3, dtype=torch.long).random_(5)
    # print(temp_input)
    # print(temp_target)
    """
    # print(target_seq.view(-1).long()) #tensor([0, 2, 2, 3])
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    print()
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

training input_seq.shape : torch.Size([3, 14, 17])
forward x.shape :torch.Size([3, 14, 17])
forward rnn hidden.shape :torch.Size([1, 3, 12])
forward rnn out.shape : torch.Size([3, 14, 12])
forward fc out.shape : torch.Size([42, 17])
training target_seq.shape :torch.Size([3, 14])

training input_seq.shape : torch.Size([3, 14, 17])
forward x.shape :torch.Size([3, 14, 17])
forward rnn hidden.shape :torch.Size([1, 3, 12])
forward rnn out.shape : torch.Size([3, 14, 12])
forward fc out.shape : torch.Size([42, 17])
training target_seq.shape :torch.Size([3, 14])

training input_seq.shape : torch.Size([3, 14, 17])
forward x.shape :torch.Size([3, 14, 17])
forward rnn hidden.shape :torch.Size([1, 3, 12])
forward rnn out.shape : torch.Size([3, 14, 12])
forward fc out.shape : torch.Size([42, 17])
training target_seq.shape :torch.Size([3, 14])

training input_seq.shape : torch.Size([3, 14, 17])
forward x.shape :torch.Size([3, 14, 17])
forward rnn hidden.shape :torch.Size([1, 3, 12])
forward rnn out.

In [0]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    print("predict character before one_hot_encode : " + str(character))
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    print("predict character after one_hot_encode : \n" + str(character))
    character = torch.from_numpy(character)
    character.to(device)

    """
    [[1]]

    [[[0. 1. 0. 0.]]]

    predict char_ind :  0
    new char : e

    [[1 0]]

    [[[0. 1. 0. 0.]
    [1. 0. 0. 0.]]]

    predict char_ind :  2
    new char : l

    [[1 0 2]]

    [[[0. 1. 0. 0.]
    [1. 0. 0. 0.]
    [0. 0. 1. 0.]]]

    predict char_ind :  2
    new char : l

    [[1 0 2 2]]

    [[[0. 1. 0. 0.]
    [1. 0. 0. 0.]
    [0. 0. 1. 0.]
    [0. 0. 1. 0.]]]

    predict char_ind :  3
    new char : o

    """

    """
    How does RNN architecture actually look like?
    for example, here hidden dim is 12 and length of input_seq is only 4
    then what do we feed to other cells (like cell#5~ cell#11)

    -> I thought hidden_dim is somewhat related to input_seq but it's actullay not.
    length of input_seq will decide how many LSTM cell will be there. 
    and each LSTM cell has #<hidden_dim> of units inside.  
    
    """

    out, hidden = model(character)
    print("predict out.shape :" + str(out.shape))
    print("predict hidden.shape :" + str(hidden.shape))
    print("predict out[-1].shape :" + str(out[-1].shape))

    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()
    print("predict char_ind : " , char_ind)
    print("prediction done")
    print("")
    return int2char[char_ind], hidden

In [0]:
def sample(model, out_len, start='e'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    print("sample chars : :",chars)
    size = out_len - len(chars)
    print("sample size : " , size)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        print("new char :" , char)
        # initially chars has start(ex. 'hey') so it just need to add predicted chars at the end of <start>
        chars.append(char)

    print("last chars :" , chars)
    return ''.join(chars)

In [0]:

# print(sample(model, 6, 'hel'))
# print()
# print()
# print(sample(model, 6, 'he'))
# print()
# print()
print(sample(model, 7, 'h'))


sample chars : : ['h']
sample size :  6
predict character before one_hot_encode : [[7]]
predict character after one_hot_encode : 
[[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
forward x.shape :torch.Size([1, 1, 17])
forward rnn hidden.shape :torch.Size([1, 1, 12])
forward rnn out.shape : torch.Size([1, 1, 12])
forward fc out.shape : torch.Size([1, 17])
predict out.shape :torch.Size([1, 17])
predict hidden.shape :torch.Size([1, 1, 12])
predict out[-1].shape :torch.Size([17])
predict char_ind :  4
prediction done

new char : e
predict character before one_hot_encode : [[7 4]]
predict character after one_hot_encode : 
[[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
forward x.shape :torch.Size([1, 2, 17])
forward rnn hidden.shape :torch.Size([1, 1, 12])
forward rnn out.shape : torch.Size([1, 2, 12])
forward fc out.shape : torch.Size([2, 17])
predict out.shape :torch.Size([2, 17])
predict hidden.shape :torch.Size([1,